In [ ]:
import pymc as pm
import pandas as pd
import numpy as np

df = pd.read_csv('date_colesterol.csv')
t = df['Ore_Exercitii'].values
y = df['Colesterol'].values

t_mean, t_std = t.mean(), t.std()
y_mean, y_std = y.mean(), y.std()

t_s = (t - t_mean) / t_std
y_s = (y - y_mean) / y_std

def build_model(t_data, y_data, K):
    with pm.Model() as model:
        weights = pm.Dirichlet('w', a=np.ones(K))
        
        alpha = pm.Normal('alpha', mu=0, sigma=10, shape=K)
        beta = pm.Normal('beta', mu=0, sigma=10, shape=K)
        gamma = pm.Normal('gamma', mu=0, sigma=10, shape=K)
        
        sigma = pm.HalfNormal('sigma', sigma=10, shape=K)
        X = t_data[:, None] 
        mu = alpha + beta * X + gamma * X**2
        
        obs = pm.NormalMixture('obs', w=weights, mu=mu, sigma=sigma, observed=y_data)
        
    return model

idatas = {}
models = {}
K = [3, 4, 5]

for k in K:
    model = build_model(t_s, y_s, k)
    models[k] = model
    with model:
        idata = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42)
        idatas[k] = idata
